## Homework 7
November 7, 2022<br>
Vivian Lin<br>
ID: 6043749830<br>
Dr. Mohammad Rajati

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, hamming_loss, silhouette_score
from sklearn.svm import SVC, LinearSVC
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.cluster import KMeans

<b>1. Multiclass and Multilabel Classification Using Support Vector Machines<br>
1a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data randomly as the training set.</b>

In [2]:
data = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
data

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [3]:
X_data = data.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
y_data = data[['Family', 'Genus', 'Species']]

# Choose 70% of the data randomly as training set
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(5036, 22) (5036, 3) (2159, 22) (2159, 3)


<b>1b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-label classification is to train a classifier for each label (binary relevance).<br>
1bi) Research exact match and hamming score/ loss methods for evaluating multi- label classification and use them in evaluating the classifiers in this problem</b>

According to the lecture slides:<br>
Exact match: Calculates subset accuracy (Predicted set of labels should exactly match the true set of labels).<br>
Hamming loss: Fraction of wrong labels to the total number of labels (N * L)<br>
In sklearn packages: Exact match accuracy is sklearn.metrics.accuracy_score and Hamming loss is sklearn.metrics.hamming_loss<br>
Hamming score is the complement of hamming loss, meaning it is 1 - hamming loss.<br>
Hamming score is a better metric to be compared with exact match accuracy.

<b>1bii) Train a SVM for each of the labels, using Gaussian kernels (RBF) and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both standardized and raw attributes and report the results. When both σ and λ are to be chosen using cross-validation, combinations of very small and very large λ’s and σ’s that keep the accuracy above a threshold can be used to determine the ranges for σ and λ</b>

In [4]:
# Need to select the best weight of SVM penalty and width of Gaussian kernel.
cv = StratifiedKFold(10)

# Combinations of very small and very large values
C_reg_combo = [0.001, 0.1, 1000, 100000]
gamma_combo = [0.001, 0.1, 1000, 100000]

# Runtime was very long, so instead of the suggested range in footnote, went for one below range, one in range, and one above range for both C and gamma.
C_reg_s = [0.0001, 0.001, 0.01, 0.1, 1]
C_reg_l = [100, 1000, 10000, 100000, 1000000]
gamma_s = [0.0001, 0.001, 0.01, 0.1, 1]
gamma_l = [100, 1000, 10000, 100000, 1000000]

rbf = SVC(kernel='rbf')
range_rbf_CV = GridSearchCV(rbf, param_grid={"C": C_reg_combo, "gamma": gamma_combo}, cv=cv)


Part 1: Train classifier for Family labels

In [5]:
# Train classifier for family label
range_rbf_CV.fit(X_train, y_train['Family'])
print(f"Best ranges for Family labels: {range_rbf_CV.best_params_}")

Best ranges for Family labels: {'C': 1000, 'gamma': 0.1}


In [7]:
# Range should be large for C, but small for gamma
rbf_CVf = GridSearchCV(rbf, param_grid={"C": C_reg_l, "gamma": gamma_s}, cv=cv)
rbf_CVf.fit(X_train, y_train['Family'])
print(f"Best regularization terms for Family labels: {rbf_CVf.best_params_}")

# Predict
yhat_f = rbf_CVf.predict(X_test)

Best regularization terms for Family labels: {'C': 100, 'gamma': 1}


Part 2: Train classifier for Genus labels

In [8]:
# Train classifier for genus label
range_rbf_CV.fit(X_train, y_train['Genus'])
print(f"Best ranges for Genus column: {range_rbf_CV.best_params_}")

Best ranges for Genus column: {'C': 1000, 'gamma': 0.1}


In [9]:
# Range should be large for C, but small for gamma
rbf_CVg = GridSearchCV(rbf, param_grid={"C": C_reg_l, "gamma": gamma_s}, cv=cv)
rbf_CVg.fit(X_train, y_train['Genus'])
print(f"Best regularization terms for Genus labels: {rbf_CVg.best_params_}")

# Predict
yhat_g = rbf_CVg.predict(X_test)

Best regularization terms for Genus labels: {'C': 100, 'gamma': 1}


Part 3: Train classifier for Species labels

In [10]:
# Train classifier for species label
range_rbf_CV.fit(X_train, y_train['Species'])
print(f"Best range for Species labels: {range_rbf_CV.best_params_}")

Best range for Species labels: {'C': 1000, 'gamma': 0.1}


In [11]:
# Train classifier for species label
rbf_CVs = GridSearchCV(rbf, param_grid={"C": C_reg_l, "gamma": gamma_s}, cv=cv)
rbf_CVs.fit(X_train, y_train['Species'])
print(f"Best regularization terms for Species labels: {rbf_CVs.best_params_}")

# Predict
yhat_s = rbf_CVs.predict(X_test)

Best regularization terms for Species labels: {'C': 100, 'gamma': 1}


Part 4: Report reseults based on exact match accuracy and hamming loss and score.

In [12]:
# Concatenate all three y labels
yhat_rbf = {"rbf_Family": yhat_f, "rbf_Genus": yhat_g, "rbf_Species": yhat_s}
y_test_merge = y_test.reset_index(drop=True)
yhat_rbf_df = pd.DataFrame(yhat_rbf)
master_rbf = pd.concat([y_test_merge, yhat_rbf_df], axis=1)

master_rbf['Triple'] = master_rbf[['Family', 'Genus', 'Species']].apply('-'.join, axis=1)
master_rbf['rbf_Triple'] = master_rbf[['rbf_Family', 'rbf_Genus', 'rbf_Species']].apply('-'.join, axis=1)
master_rbf

,Family,Genus,Species,rbf_Family,rbf_Genus,rbf_Species,Triple,rbf_Triple
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCinerascens,Hylidae,Hypsiboas,HypsiboasCinerascens,Hylidae-Hypsiboas-HypsiboasCinerascens,Hylidae-Hypsiboas-HypsiboasCinerascens
2,Hylidae,Hypsiboas,HypsiboasCordobae,Hylidae,Hypsiboas,HypsiboasCordobae,Hylidae-Hypsiboas-HypsiboasCordobae,Hylidae-Hypsiboas-HypsiboasCordobae
3,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
4,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
...,...,...,...,...,...,...,...,...
2154,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
2155,Leptodactylidae,Adenomera,AdenomeraAndre,Leptodactylidae,Adenomera,AdenomeraAndre,Leptodactylidae-Adenomera-AdenomeraAndre,Leptodactylidae-Adenomera-AdenomeraAndre
2156,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
2157,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus


In [13]:
# Evaluate accuracy and hamming loss/score
acc_rbf = accuracy_score(master_rbf['Triple'], master_rbf['rbf_Triple'])
ham_rbf = hamming_loss(master_rbf['Triple'], master_rbf['rbf_Triple'])
print(f"For SVC with Gaussian kernel, the exact match accuracy is {acc_rbf}, hamming loss is {ham_rbf}, and hamming score is {1 - ham_rbf}.")

For SVC with Gaussian kernel, the exact match accuracy is 0.9884205650764243, hamming loss is 0.01157943492357573, and hamming score is 0.9884205650764243.


<b>1biii) Repeat 1(b)ii with L1-penalized SVMs (convention is to use L1 penalty with linear kernel). Remember to standardize the attributes. Determine the weight of the SVM penalty using 10 fold cross validation</b>

Part 1: Standardize attributes, define model and CV for classifier.

In [14]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [15]:
C_l1 = [0.01, 0.1, 1, 10, 100, 1000]
l1_svc = LinearSVC(penalty='l1', dual=False, multi_class='ovr')
l1_svc_CV = GridSearchCV(l1_svc, param_grid={"C": C_l1}, cv=cv)

Part 2: Train L1-penalized with cross-validation for Family labels

In [16]:
l1_svc_CV.fit(X_train_sc, y_train['Family'])
print(f"Best regularization terms for Family labels: {l1_svc_CV.best_params_}")

# Predict
yhat_svcf = l1_svc_CV.predict(X_test_sc)

/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/s

Best regularization terms for Family labels: {'C': 1}


/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Part 3: Train L1-penalized with cross-validation for Genus labels

In [17]:
l1_svc_CV.fit(X_train_sc, y_train['Genus'])
print(f"Best regularization terms for Genus labels: {l1_svc_CV.best_params_}")

# Predict
yhat_svcg = l1_svc_CV.predict(X_test_sc)

/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/s

Best regularization terms for Genus labels: {'C': 10}


/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Part 4: Train L1-penalized with cross-validation for Species labels

In [18]:
l1_svc_CV.fit(X_train_sc, y_train['Species'])
print(f"Best regularization terms for Species labels: {l1_svc_CV.best_params_}")

# Predict
yhat_svcs = l1_svc_CV.predict(X_test_sc)

/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/s

Best regularization terms for Species labels: {'C': 10}


/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Part 5: Report results based on exact match accuracy and hamminhg loss.

In [19]:
# Concatenate all three y labels
yhat_lsvc = {"lsvc_Family": yhat_svcf, "lsvc_Genus": yhat_svcg, "lsvc_Species": yhat_svcs}
yhat_lsvc_df = pd.DataFrame(yhat_lsvc)
master_lsvc = pd.concat([y_test_merge, yhat_lsvc_df], axis=1)

master_lsvc['Triple'] = master_lsvc[['Family', 'Genus', 'Species']].apply('-'.join, axis=1)
master_lsvc['lsvc_Triple'] = master_lsvc[['lsvc_Family', 'lsvc_Genus', 'lsvc_Species']].apply('-'.join, axis=1)
master_lsvc

,Family,Genus,Species,lsvc_Family,lsvc_Genus,lsvc_Species,Triple,lsvc_Triple
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCinerascens,Hylidae,Hypsiboas,HypsiboasCinerascens,Hylidae-Hypsiboas-HypsiboasCinerascens,Hylidae-Hypsiboas-HypsiboasCinerascens
2,Hylidae,Hypsiboas,HypsiboasCordobae,Hylidae,Hypsiboas,HypsiboasCordobae,Hylidae-Hypsiboas-HypsiboasCordobae,Hylidae-Hypsiboas-HypsiboasCordobae
3,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
4,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
...,...,...,...,...,...,...,...,...
2154,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
2155,Leptodactylidae,Adenomera,AdenomeraAndre,Leptodactylidae,Adenomera,AdenomeraAndre,Leptodactylidae-Adenomera-AdenomeraAndre,Leptodactylidae-Adenomera-AdenomeraAndre
2156,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
2157,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus


In [20]:
# Evaluate accuracy and hamming loss/score
acc_lsvc = accuracy_score(master_lsvc['Triple'], master_lsvc['lsvc_Triple'])
ham_lsvc = hamming_loss(master_lsvc['Triple'], master_lsvc['lsvc_Triple'])
print(f"For linear SVC with L1 penalty, the exact match accuracy is {acc_lsvc}, hamming loss is {ham_lsvc}, and hamming score is {1 - ham_lsvc}.")

For linear SVC with L1 penalty, the exact match accuracy is 0.9129226493747105, hamming loss is 0.08707735062528948, and hamming score is 0.9129226493747105.


<b>1biv) Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.</b>

Part 1: Use SMOTE to remedy class imbalance.

In [21]:
smote = SMOTE(random_state=42)
C_l1 = [0.01, 0.1, 1, 10, 100, 1000]
bsvc = LinearSVC(penalty='l1', dual=False, multi_class='ovr')

pipe = Pipeline(steps=[('smote', smote), ('svc', bsvc)])
bsvc_CV = GridSearchCV(pipe, param_grid={"svc__C": C_l1}, cv=cv)

Part 2: Train classifier on Family labels

In [22]:
bsvc_CV.fit(X_train_sc, y_train['Family'])
print(f"Best regularization terms for Family labels: {bsvc_CV.best_params_}")

# Predict
yhat_sm_svcf = bsvc_CV.predict(X_test_sc)

/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/s

Best regularization terms for Family labels: {'svc__C': 10}


/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Part 3: Train classifier on Genus labels

In [23]:
bsvc_CV.fit(X_train_sc, y_train['Genus'])
print(f"Best regularization terms for Genus labels: {bsvc_CV.best_params_}")

# Predict
yhat_sm_svcg = bsvc_CV.predict(X_test_sc)

/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/s

Best regularization terms for Genus labels: {'svc__C': 100}


/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Part 4: Train classifier on Species labels

In [24]:
bsvc_CV.fit(X_train_sc, y_train['Species'])
print(f"Best regularization terms for Species labels: {bsvc_CV.best_params_}")

# Predict
yhat_sm_svcs = bsvc_CV.predict(X_test_sc)

/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/s

Best regularization terms for Species labels: {'svc__C': 1}


/Users/vivianlin/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Part 5: Report results

In [25]:
# Concatenate all three y labels
yhat_sm = {"sm_Family": yhat_sm_svcf, "sm_Genus": yhat_sm_svcg, "sm_Species": yhat_sm_svcs}
yhat_sm_df = pd.DataFrame(yhat_sm)
master_sm = pd.concat([y_test_merge, yhat_sm_df], axis=1)

master_sm['Triple'] = master_sm[['Family', 'Genus', 'Species']].apply('-'.join, axis=1)
master_sm['sm_Triple'] = master_sm[['sm_Family', 'sm_Genus', 'sm_Species']].apply('-'.join, axis=1)
master_sm

,Family,Genus,Species,sm_Family,sm_Genus,sm_Species,Triple,sm_Triple
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCinerascens,Hylidae,Hypsiboas,HypsiboasCinerascens,Hylidae-Hypsiboas-HypsiboasCinerascens,Hylidae-Hypsiboas-HypsiboasCinerascens
2,Hylidae,Hypsiboas,HypsiboasCordobae,Hylidae,Hypsiboas,HypsiboasCordobae,Hylidae-Hypsiboas-HypsiboasCordobae,Hylidae-Hypsiboas-HypsiboasCordobae
3,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
4,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
...,...,...,...,...,...,...,...,...
2154,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
2155,Leptodactylidae,Adenomera,AdenomeraAndre,Leptodactylidae,Adenomera,AdenomeraAndre,Leptodactylidae-Adenomera-AdenomeraAndre,Leptodactylidae-Adenomera-AdenomeraAndre
2156,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus
2157,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus,Leptodactylidae-Adenomera-AdenomeraHylaedactylus


In [26]:
# Evaluate accuracy and hamming loss/score
acc_sm = accuracy_score(master_sm['Triple'], master_sm['sm_Triple'])
ham_sm = hamming_loss(master_sm['Triple'], master_sm['sm_Triple'])
print(f"For linear SVC with SMOTE for class imbalance, the exact match accuracy is {acc_sm}, hamming loss is {ham_sm}, and hamming score is {1 - ham_sm}.")

For linear SVC with SMOTE for class imbalance, the exact match accuracy is 0.8522464103751737, hamming loss is 0.14775358962482632, and hamming score is 0.8522464103751737.


Summarize classifiers for Question 1.

In [27]:
# Only report accuracy score and hamming score (not hamming loss) because hamming score is for correctly predicted.
scores = {"rbf": [acc_rbf, 1-ham_rbf], "l1_svc": [acc_lsvc, 1-ham_lsvc], "SMOTE_l1_svc": [acc_sm, 1-ham_sm]}
scores_report = pd.DataFrame(scores, index=['exact_match', 'hamming_score'])
scores_report

,rbf,l1_svc,SMOTE_l1_svc
exact_match,0.988421,0.912923,0.852246
hamming_score,0.988421,0.912923,0.852246


<b>2. K-Means Clustering on a Multi-Class and Multi-Label Data Set<br>
Monte-Carlo Simulation: Perform procedures 50 times, and report average and standard devidation of 50 Hamming Distances calculated.</b>

<b>2a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know.<br>
2b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species.</b><br>
Outputs are combined in a single loop for 50 iterations for Monte Carlo simulation

In [28]:
# X_data and y_data already defined
print(X_data.shape, y_data.shape)

scaler = StandardScaler()
X_data_sc = scaler.fit_transform(X_data)

(7195, 22) (7195, 3)


In [29]:
ham_loss = []

for i in range(51):
    # Part a) 
    # For silhouettes, k = 1, silhouette_score = 0, so choose k from {2, 3, ... 50}
    sh_score = {}
    for k in range(2, 51):
        kmeans = KMeans(n_clusters=k, init='random')
        kmeans.fit(X_data_sc)
        sh_score[k] = silhouette_score(X_data, kmeans.labels_)

    best_k = max(sh_score, key=sh_score.get)
    print(f"[Run {i}] Best k according to Silhouette: {best_k}")

    # Model with best_k determined previously.
    kmeans_opt = KMeans(n_clusters=best_k, n_init=50)
    y_kmeans = kmeans_opt.fit_predict(X_data)
    clusters = pd.DataFrame(y_kmeans, columns=["Cluster"])

    # Part b)
    fam = pd.concat([clusters, data['Family']], axis=1)
    f_cluster_maj = {}
    for j in range(best_k):
        f_cluster_maj[j] = fam[fam['Cluster'] == j]['Family'].value_counts().idxmax()
    print(f">>>Family column, majority label for each cluster: {f_cluster_maj}")
    updated_fam = fam.replace({"Cluster": f_cluster_maj})
    updated_fam.rename(columns={"Cluster": "Fam_cluster"}, inplace=True)

    genus = pd.concat([clusters, data['Genus']], axis=1)
    g_cluster_maj = {}
    for j in range(best_k):
        g_cluster_maj[j] = genus[genus['Cluster'] == j]['Genus'].value_counts().idxmax()
    print(f">>>Genus column, majority label for each cluster: {g_cluster_maj}")
    updated_gen = genus.replace({"Cluster": g_cluster_maj})
    updated_gen.rename(columns={"Cluster": "Gen_cluster"}, inplace=True)

    spec = pd.concat([clusters, data['Species']], axis=1)
    s_cluster_maj = {}
    for j in range(best_k):
        s_cluster_maj[j] = spec[spec['Cluster'] == j]['Species'].value_counts().idxmax()
    print(f">>>Species column, majority label for each cluster: {s_cluster_maj}")
    updated_spec = spec.replace({"Cluster": s_cluster_maj})
    updated_spec.rename(columns={"Cluster": "Spec_cluster"}, inplace=True)

    master_df = pd.concat([updated_fam, updated_gen, updated_spec], axis=1)
    master_df['Triple_label'] = master_df[['Family', 'Genus', 'Species']].apply("-".join, axis=1)
    master_df['Triple_maj'] = master_df[['Fam_cluster', 'Gen_cluster', 'Spec_cluster']].apply("-".join, axis=1)

    ham_loss.append(hamming_loss(master_df['Triple_label'], master_df['Triple_maj']))


[Run 0] Best k according to Silhouette: 4
>>>Family column, majority label for each cluster: {0: 'Leptodactylidae', 1: 'Hylidae', 2: 'Dendrobatidae', 3: 'Hylidae'}
>>>Genus column, majority label for each cluster: {0: 'Adenomera', 1: 'Hypsiboas', 2: 'Ameerega', 3: 'Hypsiboas'}
>>>Species column, majority label for each cluster: {0: 'AdenomeraHylaedactylus', 1: 'HypsiboasCordobae', 2: 'Ameeregatrivittata', 3: 'HypsiboasCinerascens'}
[Run 1] Best k according to Silhouette: 4
>>>Family column, majority label for each cluster: {0: 'Hylidae', 1: 'Leptodactylidae', 2: 'Dendrobatidae', 3: 'Hylidae'}
>>>Genus column, majority label for each cluster: {0: 'Hypsiboas', 1: 'Adenomera', 2: 'Ameerega', 3: 'Hypsiboas'}
>>>Species column, majority label for each cluster: {0: 'HypsiboasCinerascens', 1: 'AdenomeraHylaedactylus', 2: 'Ameeregatrivittata', 3: 'HypsiboasCordobae'}
[Run 2] Best k according to Silhouette: 4
>>>Family column, majority label for each cluster: {0: 'Hylidae', 1: 'Hylidae', 2: 'Le

<b>2c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance, Hamming score, and Hamming loss between the true labels and the labels assigned by clusters.</b>

In [30]:
ham_score = list(map(lambda x: 1 - x, ham_loss))
ham_dist = list(map(lambda x: x * X_data_sc.shape[0], ham_loss))

In [31]:
print(f"Average Hamming distance: {sum(ham_dist) / len(ham_dist)}")
print(f"Average Hamming score: {sum(ham_score) / len(ham_score)}")
print(f"Average Hamming loss: {sum(ham_loss) / len(ham_loss)}")


Average Hamming distance: 1688.2156862745098
Average Hamming score: 0.7653626565289077
Average Hamming loss: 0.2346373434710923


<b>3. ISLR 12.6.2</b><br>
Please see PDF for answers.

References<br>
1) https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html<br>
2) https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC<br>
3) https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html<br>
4) https://medium.com/@sametgirgin/k-means-clustering-model-in-6-steps-with-python-dfe95e5a5fac<br>
5) https://stackoverflow.com/questions/50949332/silhouette-coefficient-of-hac-if-k-1<br>
6) https://datascienceparichay.com/article/most-frequent-value-in-a-pandas-column/<br>
7) https://www.statology.org/hamming-distance-python/<br>
8) https://en.wikipedia.org/wiki/Hamming_distance<br>
9) http://www.econ.upf.edu/~michael/stanford/maeb7.pdf<br>